# AutoSDF evaluation

In [1]:
import trimesh
import numpy as np
import os
import pandas as pd
from sklearn.metrics import pairwise_distances
from typing import List, Dict

# Define input and output directories
input_folder = r"/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3"
results_folder = r"/home/student/Documents/Data/AutoSDF_results/AutoSDF_results_0"
output_csv = r"/home/student/Documents/Data/Evaluation_results/evaluatie.csv"


In [2]:
# Berekeningen
def calculate_iou(mesh1: trimesh.Trimesh, mesh2: trimesh.Trimesh) -> float:
    try:
        intersection = mesh1.intersection(mesh2)
        union = mesh1.union(mesh2)
        return intersection.volume / union.volume if union.volume > 0 else 0
    except Exception:
        return -1

def chamfer_distance(mesh1: trimesh.Trimesh, mesh2: trimesh.Trimesh, num_samples: int = 10000) -> float:
    points1 = mesh1.sample(num_samples)
    points2 = mesh2.sample(num_samples)
    
    distances1 = pairwise_distances(points1, points2)
    distances2 = pairwise_distances(points2, points1)
    
    chamfer_dist1 = np.mean(np.min(distances1, axis=1))
    chamfer_dist2 = np.mean(np.min(distances2, axis=1))
    
    return chamfer_dist1 + chamfer_dist2

def normal_consistency(mesh1: trimesh.Trimesh, mesh2: trimesh.Trimesh, num_samples: int = 10000) -> float:
    points1, face_indices1 = mesh1.sample(num_samples, return_index=True)
    points2, face_indices2 = mesh2.sample(num_samples, return_index=True)

    normals1 = mesh1.face_normals[face_indices1]
    normals2 = mesh2.face_normals[face_indices2]

    dot_products = np.einsum('ij,ij->i', normals1, normals2)
    
    dot_products = np.clip(dot_products, -1.0, 1.0)

    return np.mean(np.abs(dot_products))

def f_score(mesh1: trimesh.Trimesh, mesh2: trimesh.Trimesh, threshold: float = 0.01, num_samples: int = 10000) -> float:
    points1 = mesh1.sample(num_samples)
    points2 = mesh2.sample(num_samples)

    dists1 = pairwise_distances(points1, points2)
    min_dists1 = np.min(dists1, axis=1)
    
    dists2 = pairwise_distances(points2, points1)
    min_dists2 = np.min(dists2, axis=1)

    precision = np.mean(min_dists1 < threshold)

    recall = np.mean(min_dists2 < threshold)

    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


In [ ]:
# creatie csv
def split_path(path: str) -> List[str]:
    """
    Split a file path into its folder hierarchy components.
    
    Parameters:
        path (str): File path to split.
    
    Returns:
        List[str]: List of folder components.
    """
    return path.split(os.sep)

def process_meshes(input_folder: str, results_folder: str, output_csv: str) -> None:
    """
    Process all 3D meshes from the input and results folders to compute IoU and Chamfer Distance.
    
    Parameters:
        input_folder (str): Directory containing the original meshes.
        results_folder (str): Directory containing processed meshes.
        output_csv (str): Output CSV file to save comparison results.
    """
    results: List[List[str, str, float, float, float, float, float]] = []
    input_meshes: Dict[str, str] = {}
    
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith("gt.obj"):
                rel_path = os.path.relpath(os.path.join(root, file), input_folder)
                input_meshes[rel_path] = os.path.join(root, file)
    
    for rel_path, input_path in input_meshes.items():
        result_subfolder = os.path.join(results_folder, os.path.dirname(rel_path))
        if os.path.exists(result_subfolder):
            for result_root, _, result_files in os.walk(result_subfolder):
                for result_file in result_files:
                    if result_file.endswith(".obj"):
                        result_path: str = os.path.join(result_root, result_file)
                        mesh1 = trimesh.load_mesh(input_path)
                        mesh2 = trimesh.load_mesh(result_path)
                        
                        iou = calculate_iou(mesh1, mesh2)
                        chamfer_dist = chamfer_distance(mesh1, mesh2)
                        normal_cons = normal_consistency(mesh1, mesh2)
                        f = f_score(mesh1, mesh2)
                        
                        rel_result_path = os.path.relpath(result_path, results_folder)
                        input_hierarchy = split_path(rel_path)
                        result_folder = os.path.basename(os.path.dirname(rel_result_path))
                        result_filename = os.path.basename(rel_result_path)
                        
                        results.append(input_hierarchy + [result_folder, result_filename, iou, chamfer_dist, normal_cons, f])
                        print(results[-1])
    
    max_depth = max(len(row) for row in results)
    column_names = [f"Level {i+1}" for i in range(max_depth - 4)] + ['Result Folder', 'Result File', 'IoU', 'Chamfer Distance', 'Normal Consistancy', 'f-score']
    
    df = pd.DataFrame(results, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Execute the mesh processing function
process_meshes(input_folder, results_folder, output_csv)


['gt.obj', '', '1.obj', -1, 0.05696947254018829, 0.4063489246364379, 0.33940857269111796, 0.3314]
['gt.obj', '', '0.obj', -1, 0.06243365843712444, 0.4069297797415197, 0.2727707531610775, 0.2604]
['gt.obj', '', '2.obj', -1, 0.06113887600195643, 0.40827958552108956, 0.30758280234070223, 0.2904]


ValueError: 11 columns passed, passed data had 8 columns

## Resulaten niet volledig

In [4]:
import os
import trimesh
import pandas as pd
from typing import List, Dict
import numpy as np

def process_meshes(input_folder: str, results_folder: str, output_csv: str) -> None:
    """
    Verwerkt alle 3D meshes en vergelijkt ze met resultaten indien beschikbaar.
    Als er geen resultaten zijn, wordt dit vermeld in de CSV.
    
    Parameters:
        input_folder (str): Map met originele (ground truth) meshes.
        results_folder (str): Map met de gegenereerde meshes.
        output_csv (str): Output CSV-bestand waarin de resultaten worden opgeslagen.
    """
    results: List[List[str]] = []
    input_meshes: Dict[str, str] = {}

    # Verzamel alle input meshes
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith("gt.obj"):
                rel_path = os.path.relpath(os.path.join(root, file), input_folder)
                input_meshes[rel_path] = os.path.join(root, file)

    for rel_path, input_path in input_meshes.items():
        result_subfolder = os.path.join(results_folder, os.path.dirname(rel_path))
        found_result = False  # Nieuw: bijhouden of er resultaat is gevonden

        if os.path.exists(result_subfolder):
            for result_root, _, result_files in os.walk(result_subfolder):
                for result_file in result_files:
                    if result_file.endswith(".obj"):
                        found_result = True  # Er is een resultaatbestand gevonden
                        result_path: str = os.path.join(result_root, result_file)

                        mesh1 = trimesh.load_mesh(input_path)
                        mesh2 = trimesh.load_mesh(result_path)

                        iou = calculate_iou(mesh1, mesh2)
                        chamfer_dist = chamfer_distance(mesh1, mesh2)
                        normal_cons = normal_consistency(mesh1, mesh2)
                        f = f_score(mesh1, mesh2)

                        rel_result_path = os.path.relpath(result_path, results_folder)
                        input_hierarchy = split_path(rel_path)
                        result_folder = os.path.basename(os.path.dirname(rel_result_path))
                        result_filename = os.path.basename(rel_result_path)

                        results.append(input_hierarchy + [result_folder, result_filename, iou, chamfer_dist, normal_cons, f])
                        print(results[-1])
        
        # Als geen enkel resultaatbestand is gevonden
        if not found_result:
            input_hierarchy = split_path(rel_path)
            results.append(input_hierarchy + ["geen results", "", None, None, None, None])
            print(results[-1])

    # Kolomnamen genereren afhankelijk van de inputstructuur
    max_depth = max(len(row) for row in results)
    column_names = [f"Level {i+1}" for i in range(max_depth - 4)] + ['Result Folder', 'Result File', 'IoU', 'Chamfer Distance', 'Normal Consistancy', 'f-score']

    df = pd.DataFrame(results, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f"Resultaten opgeslagen in: {output_csv}")